# Домашняя работа 4. Рекомендательные системы и Spark MLlib
Вохрамеев Михаил ИУ6-55Б

# Задание 1. Анализ датасета
- Вариант 1. Animation, Romance, Documentary
- Вариант 2. Drama, Comedy, Musical
- Вариант 3. Thriller, Sci-Fi, Adventure

⚠️ Замечание: Один фильм может принадлежать разным жанрам

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом

In [1]:
surname = "Вохрамеев"

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  2
Задача № 2:  2


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
conf = pyspark.SparkConf() \
        .set("spark.executor.memory", "1g") \
        .set("spark.executor.core", "2") \
        .setAppName("moviewRecomApp") \
        .setMaster("local[4]")

In [4]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/08 01:32:06 WARN Utils: Your hostname, MacBook-Air-4.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
25/12/08 01:32:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/08 01:32:06 WARN Utils: Your hostname, MacBook-Air-4.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
25/12/08 01:32:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level

In [5]:
spark

In [6]:
# небольшой набор данных:
small_movie_data_path = "ml-latest-small/movies.csv"
small_ratings_data_path = "ml-latest-small/ratings.csv"

# большой набор данных:
large_movie_data_path = "ml-latest/movies.csv"
large_ratings_data_path = "ml-latest/ratings.csv"


Фильмы:

In [7]:
df_movies_small = spark.read.load(small_movie_data_path, 
                            format="csv", 
                            header="true", 
                            inferSchema="true", 
                            sep=",")
print("Общее количество фильмов из маленького набора:", df_movies_small.count())
df_movies_small.show(5)

Общее количество фильмов из маленького набора: 9742
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows


In [8]:
df_movies_large = spark.read.load(large_movie_data_path, 
                            format="csv", 
                            header="true", 
                            inferSchema="true", 
                            sep=",")
print("Общее количество фильмов из большого набора:", df_movies_large.count())
df_movies_large.show(5)

Общее количество фильмов из большого набора: 86537
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows


Рейтинги:

In [9]:
df_ratings_small = spark.read.load(small_ratings_data_path,          
                             format="csv", 
                             header="true", 
                             inferSchema="true", 
                             sep=",")
print("Общее количество рейтингов из малого набора:", df_ratings_small.count())
df_ratings_small.show(5)

Общее количество рейтингов из малого набора: 100836
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows


In [10]:
df_ratings_large = spark.read.load(large_ratings_data_path,          
                             format="csv", 
                             header="true", 
                             inferSchema="true", 
                             sep=",")
print("Общее количество рейтингов из большого набора:", df_ratings_large.count())
df_ratings_large.show(5)

Общее количество рейтингов из большого набора: 33832162
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
|     1|    110|   4.0|1225865086|
|     1|    158|   4.0|1225733503|
|     1|    260|   4.5|1225735204|
|     1|    356|   5.0|1225735119|
+------+-------+------+----------+
only showing top 5 rows


In [11]:
df_ratings_small.rdd.getNumPartitions()

1

In [12]:
df_ratings_large.rdd.getNumPartitions()

7

Repartition the rdd:

In [13]:
df_ratings_small = df_ratings_small.repartition(4)
df_ratings_small.rdd.getNumPartitions()

4

Display the number of items in each partition:

In [14]:
def count_items(items):
    counts = 0
    for item in items:
        counts += 1
    yield counts

df_ratings_small.rdd \
    .mapPartitions(count_items) \
    .collect()

[25209, 25209, 25209, 25209]

In [15]:
import pyspark.sql.functions as F
from pyspark.sql.functions import split, explode, col, count, row_number, avg

#### Задача 1

Разбиваем строку genres на отдельные жанры и сразу фильтруем:

In [16]:
target_genres = ["Drama", "Comedy", "Musical"]
movies_genres_small = df_movies_small.withColumn("genre", explode(split(col("genres"), "\\|"))).filter(col("genre").isin(target_genres)).drop("genres")

movies_genres_small.show(10)

+-------+--------------------+------+
|movieId|               title| genre|
+-------+--------------------+------+
|      1|    Toy Story (1995)|Comedy|
|      3|Grumpier Old Men ...|Comedy|
|      4|Waiting to Exhale...|Comedy|
|      4|Waiting to Exhale...| Drama|
|      5|Father of the Bri...|Comedy|
|      7|      Sabrina (1995)|Comedy|
|     11|American Presiden...|Comedy|
|     11|American Presiden...| Drama|
|     12|Dracula: Dead and...|Comedy|
|     14|        Nixon (1995)| Drama|
+-------+--------------------+------+
only showing top 10 rows


In [17]:
movies_genres_large = df_movies_large.withColumn("genre", explode(split(col("genres"), "\\|"))).filter(col("genre").isin(target_genres)).drop("genres")

movies_genres_large.show(10)

+-------+--------------------+------+
|movieId|               title| genre|
+-------+--------------------+------+
|      1|    Toy Story (1995)|Comedy|
|      3|Grumpier Old Men ...|Comedy|
|      4|Waiting to Exhale...|Comedy|
|      4|Waiting to Exhale...| Drama|
|      5|Father of the Bri...|Comedy|
|      7|      Sabrina (1995)|Comedy|
|     11|American Presiden...|Comedy|
|     11|American Presiden...| Drama|
|     12|Dracula: Dead and...|Comedy|
|     14|        Nixon (1995)| Drama|
+-------+--------------------+------+
only showing top 10 rows


Считаем количество фильмов в каждом жанре из target_genres:

In [18]:
genre_counts_small = (movies_genres_small.groupBy("genre").agg(count("movieId").alias("movie_count")).orderBy(col("movie_count").desc()))
genre_counts_small.show()

+-------+-----------+
|  genre|movie_count|
+-------+-----------+
|  Drama|       4361|
| Comedy|       3756|
|Musical|        334|
+-------+-----------+



In [19]:
genre_counts_large = (movies_genres_large.groupBy("genre").agg(count("movieId").alias("movie_count")).orderBy(col("movie_count").desc()))
genre_counts_large.show()

+-------+-----------+
|  genre|movie_count|
+-------+-----------+
|  Drama|      33681|
| Comedy|      22829|
|Musical|       1059|
+-------+-----------+



#### Задача 2

Добавим столбец с кол-вом рейтингов:

In [20]:
ratings_stats_small = df_ratings_small.groupBy("movieId").agg(count("*").alias("ratings_num"))
movies_stats_small = movies_genres_small.join(ratings_stats_small, "movieId", "left")
movies_stats_small = movies_stats_small.fillna(0, subset=["ratings_num"])

movies_stats_small.show(10)

+-------+--------------------+------+-----------+
|movieId|               title| genre|ratings_num|
+-------+--------------------+------+-----------+
|      1|    Toy Story (1995)|Comedy|        215|
|      3|Grumpier Old Men ...|Comedy|         52|
|      4|Waiting to Exhale...|Comedy|          7|
|      4|Waiting to Exhale...| Drama|          7|
|      5|Father of the Bri...|Comedy|         49|
|      7|      Sabrina (1995)|Comedy|         54|
|     11|American Presiden...|Comedy|         70|
|     11|American Presiden...| Drama|         70|
|     12|Dracula: Dead and...|Comedy|         19|
|     14|        Nixon (1995)| Drama|         18|
+-------+--------------------+------+-----------+
only showing top 10 rows


In [21]:
ratings_stats_large = df_ratings_large.groupBy("movieId").agg(count("*").alias("ratings_num"))
movies_stats_large = movies_genres_large.join(ratings_stats_large, "movieId", "left")
movies_stats_large = movies_stats_large.fillna(0, subset=["ratings_num"])

movies_stats_large.show(10)

+-------+--------------------+------+-----------+
|movieId|               title| genre|ratings_num|
+-------+--------------------+------+-----------+
|     12|Dracula: Dead and...|Comedy|       4715|
|      1|    Toy Story (1995)|Comedy|      76813|
|     16|       Casino (1995)| Drama|      23932|
|      3|Grumpier Old Men ...|Comedy|      15820|
|      5|Father of the Bri...|Comedy|      15801|
|      4|Waiting to Exhale...|Comedy|       3028|
|      4|Waiting to Exhale...| Drama|       3028|
|      7|      Sabrina (1995)|Comedy|      15596|
|     11|American Presiden...|Comedy|      20256|
|     11|American Presiden...| Drama|      20256|
+-------+--------------------+------+-----------+
only showing top 10 rows


Найдем первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра из target_genres:

In [22]:
from pyspark.sql.window import Window

In [23]:
window_spec = Window.partitionBy("genre").orderBy(col("ratings_num").desc())
top_rate_num_movies_small = movies_stats_small.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
top_rate_num_movies_small.select("genre", "title", "ratings_num").show(30, truncate=False)

+-------+-------------------------------------------+-----------+
|genre  |title                                      |ratings_num|
+-------+-------------------------------------------+-----------+
|Comedy |Forrest Gump (1994)                        |329        |
|Comedy |Pulp Fiction (1994)                        |307        |
|Comedy |Toy Story (1995)                           |215        |
|Comedy |Aladdin (1992)                             |183        |
|Comedy |Fargo (1996)                               |181        |
|Comedy |True Lies (1994)                           |178        |
|Comedy |Back to the Future (1985)                  |171        |
|Comedy |Shrek (2001)                               |170        |
|Comedy |Men in Black (a.k.a. MIB) (1997)           |165        |
|Comedy |Ace Ventura: Pet Detective (1994)          |161        |
|Drama  |Forrest Gump (1994)                        |329        |
|Drama  |Shawshank Redemption, The (1994)           |317        |
|Drama  |P

In [24]:
window_spec = Window.partitionBy("genre").orderBy(col("ratings_num").desc())
top_rate_num_movies_large = movies_stats_large.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
top_rate_num_movies_large.select("genre", "title", "ratings_num").show(30, truncate=False)

+-------+-------------------------------------------------------------+-----------+
|genre  |title                                                        |ratings_num|
+-------+-------------------------------------------------------------+-----------+
|Comedy |Forrest Gump (1994)                                          |113581     |
|Comedy |Pulp Fiction (1994)                                          |108756     |
|Comedy |Toy Story (1995)                                             |76813      |
|Comedy |Back to the Future (1985)                                    |67777      |
|Comedy |Fargo (1996)                                                 |61977      |
|Comedy |Shrek (2001)                                                 |58529      |
|Comedy |Aladdin (1992)                                               |55791      |
|Comedy |True Lies (1994)                                             |52789      |
|Comedy |Pirates of the Caribbean: The Curse of the Black Pearl (2003)|50868

#### Задача 3

Найдем первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра из target_genres:

In [25]:
window_spec = Window.partitionBy("genre").orderBy(col("ratings_num").asc())
lowest_rate_num_movies_small = movies_stats_small.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
lowest_rate_num_movies_small.select("genre", "title", "ratings_num").show(30, truncate=False)

+-------+------------------------------------------------------+-----------+
|genre  |title                                                 |ratings_num|
+-------+------------------------------------------------------+-----------+
|Comedy |P.S. I Love You (2007)                                |11         |
|Comedy |Dumb and Dumberer: When Harry Met Lloyd (2003)        |11         |
|Comedy |Date Night (2010)                                     |11         |
|Comedy |Kick-Ass 2 (2013)                                     |11         |
|Comedy |My Blue Heaven (1990)                                 |11         |
|Comedy |William Shakespeare's A Midsummer Night's Dream (1999)|11         |
|Comedy |Nine to Five (a.k.a. 9 to 5) (1980)                   |11         |
|Comedy |Now You See Me 2 (2016)                               |11         |
|Comedy |How the Grinch Stole Christmas! (1966)                |11         |
|Comedy |Breaking Away (1979)                                  |11         |

In [26]:
lowest_rate_num_movies_large = movies_stats_large.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
lowest_rate_num_movies_large.select("genre", "title", "ratings_num").show(30, truncate=False)

+-------+----------------------------------------------------------------------------------------+-----------+
|genre  |title                                                                                   |ratings_num|
+-------+----------------------------------------------------------------------------------------+-----------+
|Comedy |Animal (1977)                                                                           |11         |
|Comedy |Clubland (a.k.a. Introducing the Dwights) (2007)                                        |11         |
|Comedy |House Party (2023)                                                                      |11         |
|Comedy |Kid from Brooklyn, The (1946)                                                           |11         |
|Comedy |Embassy (2013)                                                                          |11         |
|Comedy |Two Tars (1928)                                                                         |11         |
|

#### Задача 4

Добавим столбец со средним рейтингом в movies_stats:

In [27]:
avg_ratings_stats_small = df_ratings_small.groupBy("movieId").agg(avg("rating").alias("avg_rating"))
movies_stats_small = movies_stats_small.join(avg_ratings_stats_small, "movieId", "left")

movies_stats_small.show(10)

+-------+--------------------+------+-----------+------------------+
|movieId|               title| genre|ratings_num|        avg_rating|
+-------+--------------------+------+-----------+------------------+
|      1|    Toy Story (1995)|Comedy|        215|3.9209302325581397|
|      3|Grumpier Old Men ...|Comedy|         52|3.2596153846153846|
|      4|Waiting to Exhale...|Comedy|          7| 2.357142857142857|
|      4|Waiting to Exhale...| Drama|          7| 2.357142857142857|
|      5|Father of the Bri...|Comedy|         49|3.0714285714285716|
|      7|      Sabrina (1995)|Comedy|         54| 3.185185185185185|
|     11|American Presiden...|Comedy|         70|3.6714285714285713|
|     11|American Presiden...| Drama|         70|3.6714285714285713|
|     12|Dracula: Dead and...|Comedy|         19|2.4210526315789473|
|     14|        Nixon (1995)| Drama|         18|3.8333333333333335|
+-------+--------------------+------+-----------+------------------+
only showing top 10 rows


In [28]:
avg_ratings_stats_large = df_ratings_large.groupBy("movieId").agg(avg("rating").alias("avg_rating"))
movies_stats_large = movies_stats_large.join(avg_ratings_stats_large, "movieId", "left")

movies_stats_large.show(10)

+-------+--------------------+------+-----------+------------------+
|movieId|               title| genre|ratings_num|        avg_rating|
+-------+--------------------+------+-----------+------------------+
|      1|    Toy Story (1995)|Comedy|      76813|3.8935076093890357|
|     12|Dracula: Dead and...|Comedy|       4715| 2.683881230116649|
|     16|       Casino (1995)| Drama|      23932| 3.829663212435233|
|      3|Grumpier Old Men ...|Comedy|      15820|3.1712705436156763|
|      5|Father of the Bri...|Comedy|      15801|3.0769571546104677|
|      4|Waiting to Exhale...|Comedy|       3028|2.8683949801849407|
|      4|Waiting to Exhale...| Drama|       3028|2.8683949801849407|
|      7|      Sabrina (1995)|Comedy|      15596| 3.372339061297769|
|     11|American Presiden...|Comedy|      20256|3.6587677725118484|
|     11|American Presiden...| Drama|      20256|3.6587677725118484|
+-------+--------------------+------+-----------+------------------+
only showing top 10 rows


Найдем первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра из target_genres:

In [29]:
window_spec = Window.partitionBy("genre").orderBy(col("avg_rating").desc())
top_avg_rate_movies_small = movies_stats_small.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
top_avg_rate_movies_small.select("genre", "title", "ratings_num", "avg_rating").show(30, truncate=False)

+-------+---------------------------------------------------------------------------+-----------+------------------+
|genre  |title                                                                      |ratings_num|avg_rating        |
+-------+---------------------------------------------------------------------------+-----------+------------------+
|Comedy |His Girl Friday (1940)                                                     |14         |4.392857142857143 |
|Comedy |It Happened One Night (1934)                                               |14         |4.321428571428571 |
|Comedy |Philadelphia Story, The (1940)                                             |29         |4.310344827586207 |
|Comedy |Living in Oblivion (1995)                                                  |13         |4.3076923076923075|
|Comedy |Harold and Maude (1971)                                                    |26         |4.288461538461538 |
|Comedy |Dr. Strangelove or: How I Learned to Stop Worrying and 

In [30]:
top_avg_rate_movies_large = movies_stats_large.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
top_avg_rate_movies_large.select("genre", "title", "ratings_num", "avg_rating").show(30, truncate=False)

+-------+---------------------------------------------------------------------------+-----------+------------------+
|genre  |title                                                                      |ratings_num|avg_rating        |
+-------+---------------------------------------------------------------------------+-----------+------------------+
|Comedy |James Acaster: Cold Lasagne Hate Myself 1999 (2020)                        |19         |4.421052631578948 |
|Comedy |National Theatre Live: Fleabag (2019)                                      |16         |4.40625           |
|Comedy |Come From Away (2021)                                                      |19         |4.342105263157895 |
|Comedy |Parasite (2019)                                                            |12399      |4.3299459633841435|
|Comedy |Kumbalangi Nights (2019)                                                   |18         |4.305555555555555 |
|Comedy |Adventure Time: Elements (2017)                        

#### Задача 5

Найдем первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра из target_genres:

In [31]:
window_spec = Window.partitionBy("genre").orderBy(col("avg_rating").asc())
lowest_avg_rate_movies_small = movies_stats_small.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
lowest_avg_rate_movies_small.select("genre", "title", "ratings_num", "avg_rating").show(30, truncate=False)

+-------+---------------------------------------------------------------------------+-----------+------------------+
|genre  |title                                                                      |ratings_num|avg_rating        |
+-------+---------------------------------------------------------------------------+-----------+------------------+
|Comedy |Problem Child (1990)                                                       |12         |1.5833333333333333|
|Comedy |Flintstones in Viva Rock Vegas, The (2000)                                 |12         |1.625             |
|Comedy |Beethoven (1992)                                                           |11         |1.7727272727272727|
|Comedy |Stop! Or My Mom Will Shoot (1992)                                          |11         |1.7727272727272727|
|Comedy |Dungeons & Dragons (2000)                                                  |12         |1.8333333333333333|
|Comedy |Sister Act 2: Back in the Habit (1993)                 

In [32]:
lowest_avg_rate_movies_large = movies_stats_large.filter(col("ratings_num") > 10).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 10).drop("rank")
lowest_avg_rate_movies_large.select("genre", "title", "ratings_num", "avg_rating").show(30, truncate=False)

+-------+------------------------------------------------------------------------+-----------+------------------+
|genre  |title                                                                   |ratings_num|avg_rating        |
+-------+------------------------------------------------------------------------+-----------+------------------+
|Comedy |Kartoffelsalat - Nicht Fragen! (2015)                                   |11         |0.8181818181818182|
|Comedy |Hip Hop Witch, Da (2000)                                                |31         |0.8548387096774194|
|Comedy |Saving Christmas (2014)                                                 |36         |0.8611111111111112|
|Comedy |SuperBabies: Baby Geniuses 2 (2004)                                     |266        |0.8796992481203008|
|Comedy |Kidnapping, Caucasian Style (2014)                                      |17         |0.9117647058823529|
|Comedy |Beethoven's Christmas Adventure (2011)                                  |15    

# Задание 2. Коллаборативная фильтрация

- Вариант 1. По схожести пользователей
- Вариант 2. По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества

#### Задача 1:

In [33]:
train_init_small, test_small = df_ratings_small.randomSplit([0.8, 0.2], seed=42)

mean_movie_rating_small = train_init_small.agg(avg("rating")).first()[0]
print("Средний рейтинг в train_init (small):", mean_movie_rating_small)

Средний рейтинг в train_init (small): 3.5025231286795626


In [34]:
train_init_large, test_large = df_ratings_large.randomSplit([0.8, 0.2], seed=42)

mean_movie_rating_large = train_init_large.agg(avg("rating")).first()[0]
print("Средний рейтинг в train_init (large):", mean_movie_rating_large)

Средний рейтинг в train_init (large): 3.5425402477347534


In [35]:
from pyspark.ml.evaluation import RegressionEvaluator

In [36]:
df_test_pred_bl_small = test_small.withColumn("prediction", F.lit(mean_movie_rating_small))
df_test_pred_bl_small.show(5)

+------+-------+------+---------+------------------+
|userId|movieId|rating|timestamp|        prediction|
+------+-------+------+---------+------------------+
|     1|    736|   3.0|964982653|3.5025231286795626|
|     1|   1210|   5.0|964980499|3.5025231286795626|
|     1|   1258|   3.0|964983414|3.5025231286795626|
|     1|   1625|   5.0|964983504|3.5025231286795626|
|     1|   2005|   5.0|964981710|3.5025231286795626|
+------+-------+------+---------+------------------+
only showing top 5 rows


In [37]:
df_test_pred_bl_large = test_large.withColumn("prediction", F.lit(mean_movie_rating_large))
df_test_pred_bl_large.show(5)

+------+-------+------+----------+------------------+
|userId|movieId|rating| timestamp|        prediction|
+------+-------+------+----------+------------------+
|     1|    158|   4.0|1225733503|3.5425402477347534|
|     1|    596|   4.0|1225733524|3.5425402477347534|
|     1|   1049|   3.0|1225734079|3.5425402477347534|
|     1|   1214|   4.0|1225736426|3.5425402477347534|
|     1|   1909|   3.0|1225733717|3.5425402477347534|
+------+-------+------+----------+------------------+
only showing top 5 rows


In [38]:
eval_rmse_small = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_baseline_small = eval_rmse_small.evaluate(df_test_pred_bl_small)
print("RMSE baseline (small):", rmse_baseline_small)

RMSE baseline (small): 1.0458337508312192


In [39]:
eval_rmse_large = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_baseline_large = eval_rmse_large.evaluate(df_test_pred_bl_large)
print("RMSE baseline (large):", rmse_baseline_large)

RMSE baseline (large): 1.0639665452957485


#### Задача 2:

In [40]:
import sys

In [41]:
sys.path.insert(0, "/lib/python/recommend")

In [42]:
from lib.python.recommend.itemrecom import ItemBasedRecommend

In [43]:
item_based_small = ItemBasedRecommend(spark)
item_based_large = ItemBasedRecommend(spark)

Обучение:

Рассчитаем схожесть между объектами (в данном случае фильмами):

In [44]:
item_based_small.train(train_init_small, user_column_name="userId", item_column_name="movieId")

В данном задании я не запускаю на large датасете, т.к у меня нет памяти на ноутбуке и spark падает с ошибкой no space left on the device...

Предсказание:

In [45]:
USER = 1
TOP_N_RATINGS = 10

In [46]:
df_recom_small = item_based_small.recommend(user_ids=[USER], top_N_ratings=TOP_N_RATINGS, grouped=False, partition_num=4)
df_recom_small.show()

+----+-----+-----------------+
|user| item|      rating_pred|
+----+-----+-----------------+
|   1|  333|5.000000000000001|
|   1|  953|5.000000000000001|
|   1|33794|5.000000000000001|
|   1| 1129|5.000000000000001|
|   1| 1179|5.000000000000001|
|   1| 1244|5.000000000000001|
|   1| 1614|5.000000000000001|
|   1| 1769|5.000000000000001|
|   1| 2324|5.000000000000001|
|   1| 2352|5.000000000000001|
+----+-----+-----------------+



Тест:

Сбор ID пользователей из тестового поднабора:

In [47]:
U_small = {el["userId"] for el in test_small[[F.col("userId")]].distinct().collect()}

Датафрейм с предсказанными рейтингами для выбранных пользователей:

In [48]:
df_recom_small = item_based_small.recommend(user_ids=U_small, grouped=False, partition_num=4)
df_recom_small.persist().count()
df_recom_small.show(5)

+----+-----+-----------------+
|user| item|      rating_pred|
+----+-----+-----------------+
|  12| 2796|5.000000000000001|
|  12| 4306|5.000000000000001|
|  12| 6377|5.000000000000001|
|  12|47970|5.000000000000001|
|  12|65585|              5.0|
+----+-----+-----------------+
only showing top 5 rows


Объединим реальные рейтинги и предсказанные:

In [49]:
from pyspark.storagelevel import StorageLevel

In [50]:
df_true_pred_small = test_small.join(df_recom_small, on=[(F.col("userId")==F.col("user")), 
                                          (F.col("movieId")==F.col("item"))])
df_true_pred_small.persist(StorageLevel.MEMORY_ONLY).count()
df_true_pred_small.show(5)

+------+-------+------+----------+----+----+-----------+
|userId|movieId|rating| timestamp|user|item|rating_pred|
+------+-------+------+----------+----+----+-----------+
|    12|   2572|   5.0|1247263689|  12|2572|        5.0|
|    12|   2581|   5.0|1247263444|  12|2581|        5.0|
|    12|   2694|   4.0|1247263398|  12|2694|        5.0|
|    12|     39|   4.0|1247264471|  12|  39|        3.5|
|    12|   1876|   3.5|1247263334|  12|1876|        3.5|
+------+-------+------+----------+----+----+-----------+
only showing top 5 rows


In [51]:
df_true_pred_small.rdd.getNumPartitions()

4

In [52]:
df_true_pred_small.rdd \
    .mapPartitions(count_items) \
    .collect()

[3185, 5322, 3750, 4046]

Рассчитаем RMSE:

In [53]:
eval_rmse_small = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="rating_pred")
rmse_colab_small = eval_rmse_small.evaluate(df_true_pred_small)
print("RMSE: (small)", rmse_colab_small)


RMSE: (small) 0.8949607352791225


# Задание 3. Факторизация матрицы

1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4

Параметры:

- Количество факторов: [5, 10, 15]
- Регуляризация: [0.001, 0.01, 0.1, 1, 10]

⚠️ Замечание: Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

#### Задача 1

In [54]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

Настройки кросс-валидации:

In [55]:
param_grid = (ParamGridBuilder()
             .addGrid(ALS.rank, [5, 10, 15])
             .addGrid(ALS.regParam, [0.001, 0.01, 0.1, 1, 10])
             .build())

Применение ALS для предсказания рейтингов:

In [56]:
eval_rmse_small.setPredictionCol("prediction") 

als_inst = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
          coldStartStrategy="drop", seed=42)

cv_small = CrossValidator(estimator=als_inst,
                    estimatorParamMaps=param_grid,
                    evaluator=eval_rmse_small,
                    numFolds=4,
                    seed=42)

Запуск обучения:

In [57]:
cv_model_small = cv_small.fit(train_init_small)
best_model_small= cv_model_small.bestModel

print(f"Best Rank (small): {best_model_small.rank}")
print(f"Best RegParam (small): {best_model_small._java_obj.parent().getRegParam()}")


25/12/08 01:35:10 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/12/08 01:35:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Best Rank (small): 10
Best RegParam (small): 0.1


не запускаю large, т.к нет памяти на ноутбуке, код падает с ошибкой

Рассчитаем RMSE:

In [58]:
preds_small = best_model_small.transform(test_small)
rmse_als_small = eval_rmse_small.evaluate(preds_small)
print(f"RMSE лучшей модели на тест (small): {rmse_als_small}")

RMSE лучшей модели на тест (small): 0.8864855016226817


#### Задача 2

In [59]:
print("Сравнение (small):")
print(f"RMSE baseline: {rmse_baseline_small}")
print(f"Colab RMSE: {rmse_colab_small}")
print(f"ALS RMSE: {rmse_als_small}")

Сравнение (small):
RMSE baseline: 1.0458337508312192
Colab RMSE: 0.8949607352791225
ALS RMSE: 0.8864855016226817


In [60]:
spark.stop()